In [ ]:
!git clone https://github.com/rachtibat/zennit-crp
!pip install ./zennit-crp

Cloning into 'zennit-crp'...
remote: Enumerating objects: 516, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 516 (delta 52), reused 47 (delta 41), pack-reused 431 (from 1)
Receiving objects: 100% (516/516), 18.36 MiB | 6.32 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Processing ./zennit-crp
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
ERROR: Cannot install torchvision==0.20.1+cu1

In [ ]:
!pip install torch torchvision
!pip install matplotlib numpy pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install 'zennit<=0.4.6'

  Using cached zennit-0.4.6-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

sys.path.append('/content/zennitcrp/crp')
sys.path.append('/content/zennitcrp')

redémarreret renommer zennitcrp


In [ ]:
import sys
sys.path.append('/content/zennitcrp/crp')
sys.path.append('/content/zennitcrp')

In [ ]:
import crp

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

from zennit.composites import EpsilonPlusFlat
from zennit.canonizers import SequentialMergeBatchNorm
from zennitcrp.crp.attribution import CondAttribution

import torchvision.transforms as T
from PIL import Image
import os
import torch
from torchvision.models.vgg import vgg16_bn
import json

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = vgg16_bn(True).to(device)
model.eval()

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:04<00:00, 121MB/s]


In [ ]:
CLASSE_PREDICTED = "classe_predicted"
PROBABILITY = "probability"
FEATURES = "features"

In [ ]:
features_per_layer = {
    0: 64,
    3: 64,
    7: 128,
    10: 128,
    14: 256,
    17: 256,
    20: 256,
    24: 512,
    27: 512,
    30: 512,
    34: 512,
    37: 512,
    40: 512
}

In [ ]:
def compute_feature_importance(model, input_tensor, layer_idx, num_features, pred_class):
    """
    Calcule l'importance de chaque feature d'une couche donnée pour une classe prédite.

    Arguments :
    - model : le modèle VGG16
    - input_tensor : l'image d'entrée sous forme de tenseur
    - layer_idx : l'index de la couche (ex : 40)
    - num_features : le nombre total de features dans cette couche (ex : 512)
    - pred_class : la classe prédite initialement

    Retourne :
    - Un dictionnaire {feature_idx : importance} trié par importance décroissante
    """
    input_tensor.requires_grad = True
    # Obtenir la probabilité originale de la classe prédite
    with torch.no_grad():
        output_original = model(input_tensor)
        probs_original = torch.nn.functional.softmax(output_original, dim=1)
        original_score = probs_original[0, pred_class].item()

    feature_importance = {}

    # Désactiver chaque feature une par une et mesurer l'impact
    for feature_idx in range(num_features):
        def zero_out_feature(module, input, output, feature_idx=feature_idx):
            output[:, feature_idx, :, :] = 0  # Désactiver la feature
            return output

        # Ajouter un hook temporaire
        hook = model.features[layer_idx].register_forward_hook(zero_out_feature)

        # Faire une prédiction avec la feature désactivée
        with torch.no_grad():
            output_disabled = model(input_tensor)
            probs_disabled = torch.nn.functional.softmax(output_disabled, dim=1)
            new_score = probs_disabled[0, pred_class].item()

        # Supprimer le hook
        hook.remove()

        # Calcul de l'importance
        importance = original_score - new_score
        feature_importance[feature_idx] = float(importance)

        # Affichage de progression
        #print(f"Feature {feature_idx+1}/{num_features} - Importance: {importance:.4f}")

    # Trier les features par importance décroissante
    sorted_importance = dict(sorted(feature_importance.items(), key=lambda item: item[1], reverse=True))

    return sorted_importance


In [ ]:
def processPicture(global_dictionary : dict, picture_path : str = "/content/drive/My Drive/PER/data", output_heatmaps_path :str = "/content/drive/My Drive/PER/save/heatmaps"):
    local_dictionary = {}
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    image = Image.open(picture_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True

    output = model(input_tensor)
    pred_class = torch.argmax(output, dim=1).item()
    probs = torch.nn.functional.softmax(output, dim=1)

    local_dictionary[CLASSE_PREDICTED] = pred_class
    local_dictionary[PROBABILITY] = probs[0, pred_class]

    composite = EpsilonPlusFlat([SequentialMergeBatchNorm()])
    attribution = CondAttribution(model, no_param_grad=True)

    features_dict = {}
    layers_heatmaps = {}
    for layer_idx, num_features in features_per_layer.items():
        all_heatmaps = []
        num_feature_per_batch = 8
        index = 0
        borne_sup = 0
        while borne_sup != num_features:
            borne_sup = min((index+1)*num_feature_per_batch, num_features)
            conditions = [{"y": [40], "features.40": [j]} for j in range(index*num_feature_per_batch, borne_sup)]
            heatmaps, _, _, _ = attribution(input_tensor, conditions, composite)
            all_heatmaps.append(heatmaps)
            index += 1
        heatmaps = np.concatenate([heatmap.cpu().numpy() for heatmap in all_heatmaps], axis=0)
        #heatmaps = np.concatenate(all_heatmaps, axis=0)
        layers_heatmaps[layer_idx] = heatmaps

        importance_dict = compute_feature_importance(model, input_tensor, layer_idx=layer_idx, num_features=num_features, pred_class=pred_class)

        features_dict[layer_idx] = importance_dict

    # Normalisation globale sur toutes les heatmaps
    min_value = min([heatmaps.min() for heatmaps in layers_heatmaps.values()])
    max_value = max([heatmaps.max() for heatmaps in layers_heatmaps.values()])
    max_value = max(abs(min_value), abs(max_value))
    min_value = -max_value

    os.makedirs(output_heatmaps_path, exist_ok=True)
    save_path = os.path.join(output_heatmaps_path, f"{image_name}.npz")

    save_dict = {f"layer_{idx_layers}": np.array(heatmaps) for idx_layers, heatmaps in layers_heatmaps.items()}
    np.savez(save_path, **save_dict)

    local_dictionary[FEATURES] = features_dict
    global_dictionary[image_name] = local_dictionary
    return global_dictionary

In [ ]:
def tensor_to_list(obj):
    if isinstance(obj, torch.Tensor):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

In [ ]:
def create_json_from_data(data, output_filename):
    """
    Fonction qui prend un dictionnaire de données avec des informations sur des images,
    et les sauvegarde dans un fichier JSON structuré.

    :param data: Dictionnaire avec les données des images.
    :param output_filename: Nom du fichier JSON à créer.
    """
    # Créer un dictionnaire pour les données au format désiré
    image_data = {}

    for image_name, info in data.items():
        # Extraire les informations : classe, probabilité, et dictionnaire de features
        classe = info[CLASSE_PREDICTED]
        probability = info[PROBABILITY]
        features = info[FEATURES]

        # Ajouter ces informations dans le dictionnaire final
        image_data[image_name] = {
            CLASSE_PREDICTED: classe,
            PROBABILITY: probability,
            FEATURES: features
        }

    # Sauvegarder les données dans un fichier JSON
    with open(output_filename, 'w') as json_file:
        json.dump(image_data, json_file, indent=4, default=tensor_to_list)
    print(f"Le fichier JSON '{output_filename}' a été créé avec succès.")

In [ ]:
input_folder = "/content/drive/My Drive/PER/data"
global_dictionary = {}

image_paths = glob(os.path.join(input_folder, "*.jpeg"))
print(f"Nombre d'images trouvées : {len(image_paths)}")


Nombre d'images trouvées : 11


In [ ]:
avoid_images = []

In [ ]:
i = len(avoid_images)
for image_path in image_paths:
    global_dictionary = {}
    if image_path in avoid_images:
        print(f"Image {image_path} avoid")
        continue
    global_dictionary = processPicture(global_dictionary, image_path)
    avoid_images.append(image_path)
    print(avoid_images)
    create_json_from_data(global_dictionary, f"/content/drive/My Drive/PER/data{i}.json")
    i = i + 1
#create_json_from_data(global_dictionary, "/content/drive/My Drive/PER/data.json")

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


['/content/drive/My Drive/PER/data/n01443537_518.jpeg']
Le fichier JSON '/content/drive/My Drive/PER/data0.json' a été créé avec succès.
['/content/drive/My Drive/PER/data/n01443537_518.jpeg', '/content/drive/My Drive/PER/data/n01484850_19861.jpeg']
Le fichier JSON '/content/drive/My Drive/PER/data1.json' a été créé avec succès.
['/content/drive/My Drive/PER/data/n01443537_518.jpeg', '/content/drive/My Drive/PER/data/n01484850_19861.jpeg', '/content/drive/My Drive/PER/data/n01494475_7789.jpeg']
Le fichier JSON '/content/drive/My Drive/PER/data2.json' a été créé avec succès.
['/content/drive/My Drive/PER/data/n01443537_518.jpeg', '/content/drive/My Drive/PER/data/n01484850_19861.jpeg', '/content/drive/My Drive/PER/data/n01494475_7789.jpeg', '/content/drive/My Drive/PER/data/n01622779_2286.jpeg']
Le fichier JSON '/content/drive/My Drive/PER/data3.json' a été créé avec succès.
['/content/drive/My Drive/PER/data/n01443537_518.jpeg', '/content/drive/My Drive/PER/data/n01484850_19861.jpeg', 

In [ ]:
#['/content/drive/My Drive/PER/data/n01829413_46.jpeg', '/content/drive/My Drive/PER/data/n01443537_130.jpeg', '/content/drive/My Drive/PER/data/n01484850_198.jpeg', '/content/drive/My Drive/PER/data/n01494475_20.jpeg', '/content/drive/My Drive/PER/data/n01622779_172.jpeg', '/content/drive/My Drive/PER/data/n01843383_547.jpeg', '/content/drive/My Drive/PER/data/n02071294_4132.jpeg', '/content/drive/My Drive/PER/data/n02071294_4408.jpeg', '/content/drive/My Drive/PER/data/n02391049_544.jpeg', '/content/drive/My Drive/PER/data/n02437312_845.jpeg', '/content/drive/My Drive/PER/data/n02480495_162.jpeg', '/content/drive/My Drive/PER/data/n02480495_824.jpeg', '/content/drive/My Drive/PER/data/n02480855_324.jpeg', '/content/drive/My Drive/PER/data/n02486410_86.jpeg', '/content/drive/My Drive/PER/data/n02492035_742.jpeg', '/content/drive/My Drive/PER/data/n02504013_189.jpeg', '/content/drive/My Drive/PER/data/n02510455_1238.jpeg', '/content/drive/My Drive/PER/data/n02510455_190.jpeg']